<a href="https://colab.research.google.com/github/kemwe/LLM/blob/master/implemenation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install nvcc4jupyter
!pip install --upgrade diffusers
!pip install transformers accelerate safetensors

%load_ext nvcc4jupyter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 18.7 MB/s eta 0:00:00
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.30.3
    Uninstalling diffusers-0.30.3:
      Successfully uninstalled diffusers-0.30.3
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp0c3nxkfw".


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch



In [4]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

In [5]:
device = "cuda" #if torch.cuda.is_available() else "cpu"

In [1]:
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

NameError: name 'AutoModelForCausalLM' is not defined

In [ ]:
messages = [
   {"role": "user", "content": "How do chat templates work?"},
   {"role": "assistant", "content": "Chat templates help  LLMs like me generate more coherent responses by providing a structured way to organize the conversation."},
   {"role": "user", "content": "How do I use them?"},
]